# Streaks analysis

Streaks analysis is done by [Koch (20004)](https://www.climate-service-center.de/imperia/md/content/gkss/institut_fuer_kuestenforschung/ksd/paper/kochw_ieee_2004.pdf) algorithm implementation.


In [ ]:
# import needed modules
import xsar
import xsarsea
import xsarsea.gradients
import bokeh.io
bokeh.io.output_notebook()
import holoviews as hv
hv.extension('bokeh')
import xarray as xr
import numpy as np
import scipy

In [ ]:
# open a file a 100m (IW_GRDH pixel size is 10m, so we use resolution=10)
filename = xsar.get_test_file('S1A_IW_GRDH_1SDV_20170907T103020_20170907T103045_018268_01EB76_Z010.SAFE')
sar_ds = xsar.open_dataset(filename,resolution={'atrack':10,'xtrack':10}).isel(atrack=slice(20,None,None),xtrack=slice(20,None,None)) # isel to skip bad image edge

# add detrended sigma0
sar_ds['sigma0_detrend'] = xsarsea.sigma0_detrend(sar_ds.sigma0, sar_ds.incidence)

# apply land mask
land_mask = sar_ds['land_mask'].compute()
sar_ds['sigma0_detrend'] = xr.where(land_mask, np.nan, sar_ds['sigma0_detrend']).transpose(*sar_ds['sigma0_detrend'].dims).compute()


## Simple gradients analysis

Simple gradient analysis is done with [xsarsea.gradients.Gradients](../basic_api.rst#xsarsea.gradients.Gradients)

Here, we merge the gradient analysis with the original dataset. 'grad_dir' and 'grad_weigth' variables will be added.
This is easy, because the default for [xsarsea.gradients.Gradients.main_gradients](../basic_api.rst#xsarsea.gradients.Gradients.main_gradients) is to interpolate gradients on the original grid.

In [ ]:
sar_ds = xr.merge([sar_ds, xsarsea.gradients.Gradients(sar_ds['sigma0_detrend']).main_gradients()])
sar_ds

## Multi scale and multi resolution gradients analysis

[xsarsea.gradients.MultiGradients](../basic_api.rst#xsarsea.gradients.MultiGradients) can be used to compute gradients at differents scale and resolution.

Original `sigma0` pixel spacing is 100m. We can use `downscale_factors=[1,2,4]` to downscale it to 100m, 200m, and 400m, and use `windows_sizes=[160, 240, 320]` to use a sliding windows of 16km, 24km and 32km.

`window_step=1` refer to the smallest window overlaping, so there will be no overlap for the 16km window, but the overlaping for other windows will be 24-16=8km and 32-16=16km. 

with `interpolate=False`, streaks are not interpolated on the original grid, so they can't be merged into the orginal dataset (check the dimensions sizes).

In [ ]:
streaks = xsarsea.gradients.MultiGradients(sar_ds['sigma0_detrend'],downscale_factors=[1,2,4], windows_sizes=[160, 320, 480], window_step=1)

streaks_ds = streaks.main_gradients(method='max', interpolate=False)
streaks_ds

## Digging into MultiGradients

[xsarsea.gradients.MultiGradientsWiewer](../basic_api.rst#xsarsea.gradients.MultiGradientsWiewer) can be used to dig interactively with a `MultiGradient` instance.

Note that this require a live notebook, and can't be fully used from the html example page.

This is currently work in progress, more work is needed to provide an understandable figure ...

In [ ]:
xsarsea.gradients.MultiGradientsViewer(streaks).all_view

Many gradients where found. Some are probably aligned with streaks, but others certainly not.

For each position, we can compute the direction stdev, and the mean weight.

In [ ]:

streaks_ds['grad_dir_std'] = xr.zeros_like(streaks_ds['grad_dir']).isel(pol=0,downscale_factor=0,window_size=0)
streaks_ds['grad_dir_std'].values = scipy.stats.circstd(streaks_ds['grad_dir'].values,high=90, low=-90, axis=(0,1,2), nan_policy='omit')

(
    hv.QuadMesh(streaks_ds['grad_dir_std']).opts(cmap='bmy', title='dir stdev') + \
    hv.QuadMesh(np.mean(streaks_ds['grad_weight'], axis=(0,1,2))).opts(cmap='viridis', title='mean weight')
).opts(hv.opts.QuadMesh(frame_width=400, frame_height=400, tools=['hover']))


We can try to select the best weight from all computed gradients, and compare them to the one computed at 100m with 16km window size, on crosspol.

In [ ]:
# best streaks
idx = streaks_ds['grad_weight'].fillna(0).argmax(('pol','downscale_factor', 'window_size'))
best_streaks = streaks_ds.isel(pol=idx['pol'],downscale_factor=idx['downscale_factor'], window_size=idx['window_size'])
best_streaks['grad_dir_rad'] = xr.ufuncs.deg2rad(best_streaks['grad_dir'])

# streaks at 100m, crosspol
streaks_cross_100 = streaks_ds.sel(pol='VH',downscale_factor=1, window_size=160)
streaks_cross_100['grad_dir_rad'] = xr.ufuncs.deg2rad(streaks_cross_100['grad_dir'])


clim = tuple([np.nanpercentile(streaks.sigma0.sel(pol='VH'), l) for l in [5,95]])

(
    hv.Image(streaks.sigma0.sel(pol='VH')).opts(cmap='gray',clim=clim) * \
    hv.VectorField(best_streaks, vdims=['grad_dir_rad','grad_weight'], label='corrected').opts(color='r') * \
    hv.VectorField(streaks_cross_100, vdims=['grad_dir_rad','grad_weight'], label='100m 16km cr').opts(color='y') 
).opts(hv.opts.VectorField(arrow_heads=False, pivot='mid', scale=0.4, line_width=3)).opts(frame_width=800, frame_height=800)

Above figure show that some correction in the left lower corner are probaly good. But some others are probably worse than the original solution ..

More work is needed !